In [1]:
import numpy as np

In [13]:
def PSOR_American_Put(Stock, Strike, Volatility, RiskFreeRate, Dividend, Space, Expiry, Time):

    S = Stock
    T = Expiry
    vol = Volatility
    q = Dividend
    r = RiskFreeRate
    K = Strike
    omega = 1.5
    epsilon = 1e-8

    M = Time
    N = Space

    S_max = K*3
    S_min = K/10

    x_max = np.log(S_max)
    x_min = np.log(S_min)

    x, dx = np.linspace(x_min, x_max, N, retstep=True)
    t, dt = np.linspace(0, T, M, retstep=True)
    S_vec = np.exp(x)

    dx2 = dx**2
    vol2 = vol**2

    a_one = -0.25 * (vol2) * (dt / dx2) - ((r - q)  - 0.5 * vol2) * (0.25 * (dt / dx))
    a_zero = 1 + (0.5 * vol2) * (dt / dx2) + 0.5 * r * dt
    a_negative_one = -0.25 * (vol2) * (dt / dx2) + ((r - q)  - 0.5 * vol2) * (0.25 * (dt / dx))

    b_one = 0.25 * (vol2) * (dt / dx2) + ((r - q)  - 0.5 * vol2) * (0.25 * (dt / dx))
    b_zero = 1 - (0.5 * vol2) * (dt / dx2) - 0.5 * r * dt
    b_negative_one = 0.25 * (vol2) * (dt / dx2) - ((r - q) - 0.5 * vol2) * (0.25 * (dt / dx))

    B = np.diag([b_zero] * N) + np.diag([b_one] * (N - 1), 1) + np.diag([b_negative_one] * (N - 1), -1)
    V = np.zeros((M, N))

    # Initial conditions
    payoff = np.maximum(K - S_vec, 0)
    V[0,:] = payoff

    put_left_boundary = lambda i: K - np.exp(x_min - i * dx)

    for i in range(1, M):
        error = float("inf")
        v_psor = V[i-1].copy()
        d = B @ V[i-1]
        d[0] += b_negative_one * put_left_boundary(i - 1) - a_negative_one * put_left_boundary(i)
        while epsilon < error:
            v_psor_prev = v_psor.copy()
            for j in range(N):
                correction = d[j] - a_zero * v_psor[j]
                if 0 < j:
                    correction -= a_negative_one * v_psor[j - 1]
                if j < N - 1:
                    correction -= a_one * v_psor[j + 1]
                v_psor[j] += (omega / a_zero) * correction
                v_psor[j] = max(v_psor[j], payoff[j])
            diff = v_psor - v_psor_prev
            error = diff @ diff
        V[i] = v_psor

        option_price = np.interp(np.log(S), x, V[-1,:])
        upper = np.interp(np.log(S + dx), x, V[-1, :])
        lower = np.interp(np.log(S - dx), x, V[-1, :])
        delta = (upper - lower) / (2 * dx)
        

    print(f"Option price: {option_price:.6f}\n")
    print(f"Option Delta: {delta:.6f}\n")

    return V, delta


In [14]:
V, S_vec, t = PSOR_American_Put(36, 40, 0.20, 0.06, 0.00, 500, 1.0, 250)

Option price: 4.486775

Option Delta: -0.694522



ValueError: not enough values to unpack (expected 3, got 2)